In [1]:
rng = MersenneTwister(1)
using AutomotivePOMDPs
using MDPModelChecking
using GridInterpolations, StaticArrays, POMDPs, POMDPToolbox, AutoViz, AutomotiveDrivingModels, Reel
using DeepQLearning, DeepRL
using DiscreteValueIteration
using ProgressMeter, Parameters, JLD
include("util.jl")
include("masking.jl")
include("render_helpers.jl")

The method will not be callable.


In [2]:
mutable struct UrbanDriverPolicy <: Policy
    pomdp::UrbanPOMDP
    model::UrbanDriver
end

# define baseline policy 
function evaluation_loop(pomdp::UrbanPOMDP, policy::UrbanDriverPolicy; n_ep::Int64 = 1000, max_steps::Int64 = 500, rng::AbstractRNG = Base.GLOBAL_RNG)
    rewards = zeros(n_ep)
    steps = zeros(n_ep)
    violations = zeros(n_ep)
    up = FastPreviousObservationUpdater{obs_type(pomdp)}()
    for ep=1:n_ep
        policy = UrbanDriverPolicy(pomdp) # reset states
        s0 = initial_state(pomdp, rng)
        o0 = generate_o(pomdp, s0, rng)
        b0 = initialize_belief(up, o0)
        hr = HistoryRecorder(max_steps=max_steps, rng=rng)
        hist = simulate(hr, pomdp, policy, up, b0, s0);
        rewards[ep] = discounted_reward(hist)
        steps[ep] = n_steps(hist)
        # violations[ep] = sum(hist.reward_hist .< 0.)
        violations[ep] = is_crash(hist.state_hist[end])#sum(hist.reward_hist .<= -1.) #+ Int(n_steps(hist) >= max_steps)
    end
    return rewards, steps, violations
end

function  AutomotivePOMDPs.get_conflict_lanes(crosswalk::Lane, roadway::Roadway)
    # find lane intersecting with crosswalk
    cw_seg = AutomotivePOMDPs.lane_to_segment(crosswalk, roadway)
    conflict_lanes = Lane[]
    lanes = get_lanes(roadway)
    push!(lanes, roadway[LaneTag(6,1)])
    push!(lanes, roadway[LaneTag(13,1)])
    push!(lanes, roadway[LaneTag(14,1)])
    for lane in lanes
        lane_seg = AutomotivePOMDPs.lane_to_segment(lane, roadway)
        if intersects(lane_seg, cw_seg) && !(lane ∈ conflict_lanes)
            push!(conflict_lanes, lane)
        end
    end
    return conflict_lanes
end

function UrbanDriverPolicy(pomdp::UrbanPOMDP)
    route = [pomdp.env.roadway[l] for l in AutomotivePOMDPs.get_ego_route(pomdp.env)]
    intersection_entrances = Lane[pomdp.env.roadway[l] for l in [LaneTag(3,1), LaneTag(1,1), LaneTag(6, 1)]]
    if !(route[1] ∈ intersection_entrances)
        intersection = Lane[]
        intersection_exits = Lane[]
    else
        intersection_exits = get_exit_lanes(pomdp.env.roadway)
        intersection=Lane[route[1], route[2]]
    end
    navigator = RouteFollowingIDM(route=route, a_max=2.)
#     intersection_driver = StopIntersectionDriver(navigator= navigator,
#                                                 intersection=intersection,
#                                                 intersection_entrances = intersection_entrances,
#                                                 intersection_exits = intersection_exits,
#                                                 stop_delta=maximum(pomdp.env.params.crosswalk_width),
#                                                 accel_tol=0.,
#                                                 priorities = pomdp.env.priorities)
    intersection_driver = TTCIntersectionDriver(navigator = navigator,
                                                        intersection = intersection,
                                                        intersection_pos = VecSE2(pomdp.env.params.inter_x,
                                                                                  pomdp.env.params.inter_y),
                                                        stop_delta = maximum(pomdp.env.params.crosswalk_width),
                                                        accel_tol = 0.,
                                                        priorities = pomdp.env.priorities,
                                                        ttc_threshold = (pomdp.env.params.x_max - pomdp.env.params.inter_x)/pomdp.env.params.speed_limit
                                                        )
    crosswalk_drivers = Vector{CrosswalkDriver}(length(pomdp.env.crosswalks))
    # println("adding veh ", new_car.id)
    for i=1:length(pomdp.env.crosswalks)
        cw_conflict_lanes = get_conflict_lanes(pomdp.env.crosswalks[i], pomdp.env.roadway)
        crosswalk_drivers[i] = CrosswalkDriver(navigator = navigator,
                                crosswalk = pomdp.env.crosswalks[i],
                                conflict_lanes = cw_conflict_lanes,
                                intersection_entrances = intersection_entrances,
                                yield=!isempty(intersect(cw_conflict_lanes, route)),
                                stop_delta = 1.0
                                )
        # println(" yield to cw ", i, " ", crosswalk_drivers[i].yield)
    end
    model = UrbanDriver(navigator=navigator,
                        intersection_driver=intersection_driver,
                        crosswalk_drivers=crosswalk_drivers
                        )
    return UrbanDriverPolicy(pomdp, model)
end

function POMDPs.action(policy::UrbanDriverPolicy, o::Array{Float64, 1})
    s = obs_to_scene(pomdp, o)
    observe!(policy.model, s, policy.pomdp.env.roadway, EGO_ID)
    return UrbanAction(policy.model.a.a_lon)
end

function reset_policy!(policy::UrbanDriverPolicy)    
end


reset_policy! (generic function with 1 method)

In [3]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =  [VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params)

pomdp = UrbanPOMDP(env=env,
                   ego_goal = LaneTag(2, 1),
                   max_cars=1, 
                   max_peds=1, 
                   car_birth=0.7, 
                   ped_birth=0.7, 
                   obstacles=false, # no fixed obstacles
                   lidar=false,
                   pos_obs_noise = 0., # fully observable
                   vel_obs_noise = 0.);
policy = UrbanDriverPolicy(pomdp);

In [195]:
policy.model.crosswalk_drivers[3]

AutomotivePOMDPs.CrosswalkDriver
  a: AutomotivePOMDPs.LonAccelDirection
  navigator: AutomotivePOMDPs.RouteFollowingIDM
  crosswalk: AutomotiveDrivingModels.Lane
  conflict_lanes: Array{AutomotiveDrivingModels.Lane}((2,))
  intersection_entrances: Array{AutomotiveDrivingModels.Lane}((3,))
  ped_model: AutomotivePOMDPs.ConstantPedestrian
  ped_start: Float64 4.0
  stop_delta: Float64 1.0
  accel_tol: Float64 0.1
  d_tol: Float64 0.5
  yield: Bool true
  priority: Bool false
  stop: Bool false
  wait_list: Array{Int64}((0,)) Int64[]
  clear: Bool false
  debug: Bool false


In [196]:
model = policy.model.crosswalk_drivers[3]
model.yield
model.priority
policy = UrbanDriverPolicy(pomdp);
model.debug = false

false

In [197]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initial_state(pomdp, rng)
o0 = generate_o(pomdp, s0, rng)
up = FastPreviousObservationUpdater{UrbanObs}()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, policy, up, b0, s0);

  0.209522 seconds (189.99 k allocations: 6.152 MiB)


In [198]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
duration, fps, render_hist = animate_hist(pomdp, h)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmp3b2ytA", 0x0000000000000010, 2.0, nothing)

In [5]:
# evaluate resulting policy
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, policy, n_ep=10000, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

28-Jun 14:00:46:WARNING:root:replacing docs for 'AutomotivePOMDPs.ttc_check :: Tuple{AutomotivePOMDPs.TTCIntersectionDriver,Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},AutomotiveDrivingModels.Roadway,Int64}' in module 'AutomotivePOMDPs'.


557.867942 seconds (4.33 G allocations: 128.466 GiB, 13.38% gc time)
Summary for 10000 episodes: 
Average reward: 0.200 
Average # of steps: 42.048 
Average # of violations: 3.620 


In [185]:
@time for ep=1:10000
    policy = UrbanDriverPolicy(pomdp) 
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initial_state(pomdp, rng)
    o0 = generate_o(pomdp, s0, rng)
    up = FastPreviousObservationUpdater{UrbanObs}()
    b0 = initialize_belief(up, o0)
    hist2 = simulate(hr, pomdp, policy, up, b0, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Crash
  5.638042 seconds (83.47 M allocations: 2.712 GiB, 18.89% gc time)
